<a href="https://colab.research.google.com/github/AzulBarr/Aprendizaje-Automatico/blob/main/TPs/tp2/RF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Carga de librerias y modelos

In [1]:
!pip install ebooklib beautifulsoup4 pandas
!pip install stanza

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 40.4 MB/s eta 0:00:00


In [2]:
import torch
from transformers import BertTokenizer, BertModel
import numpy as np
import pandas as pd
from ebooklib import epub
import ebooklib
from bs4 import BeautifulSoup
import re
import stanza
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
model_name = "bert-base-multilingual-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)
#model.etest()

In [5]:
stanza.download("es")
nlp = stanza.Pipeline("es", processors="tokenize,pos")

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: es (Spanish) ...


KeyboardInterrupt: 

# Definición de funciones

In [135]:
def normalize_text_X(t):
    # Convertir a minúsculas y quitar puntuación
    t = t.lower()
    t = re.sub(r'[\u200b-\u200f\uFEFF]', '', t)
    t = re.sub(r"[^a-záéíóúüñ0-9' ]+", ' ', t)
    t = re.sub(r'p\s*\.?\s*e\s*\.?\s*d\s*\.?\s*d\s*\.?\s*o\s*\.?', 'peddo', t, flags=re.IGNORECASE)
    t = re.sub(r's\s*\.?\s*p\s*\.?\s*a\s*\.?\s*d\s*\.?\s*a\.?', 'spada', t, flags=re.IGNORECASE)
    t = re.sub(r'p\s*\.?\s*s\s*\.?', 'ps', t, flags=re.IGNORECASE)
    t = re.sub(r'\s+', ' ', t).strip()
    return t

In [69]:
def normalize_text_y(t):
    t = re.sub(r'[\u200b-\u200f\uFEFF]', '', t)
    t = re.sub(r"[^a-zA-ZáéíóúüñÁÉÍÓÚ0-9¿?,.' ]+", ' ', t)
    t = re.sub(r'p\s*\.?\s*e\s*\.?\s*d\s*\.?\s*d\s*\.?\s*o\s*\.?', 'PEDDO', t, flags=re.IGNORECASE)
    t = re.sub(r's\s*\.?\s*p\s*\.?\s*a\s*\.?\s*d\s*\.?\s*a\.?', 'SPADA', t, flags=re.IGNORECASE)
    t = re.sub(r'p\s*\.?\s*s\s*\.?', 'PS', t, flags=re.IGNORECASE)
    t = re.sub(r'\s+', ' ', t).strip()
    return t

In [33]:
def convertir_epub_a_pd(archivo_epub='libro.epub'):
  # Cargar el libro
  book = ebooklib.epub.read_epub(archivo_epub)

  # Lista donde se guardarán los párrafos
  parrafos = []

  # Recorremos los ítems del libro
  for item in book.get_items():
      if item.get_type() == ebooklib.ITEM_DOCUMENT:
          # Parseamos el contenido HTML
          soup = BeautifulSoup(item.get_body_content(), 'html.parser')
          # Extraemos los párrafos
          for p in soup.find_all('p'):
            #print("p:",p, 'tipo: ', type(p))
            texto = p.get_text().strip()
            #print("TEXTO:",texto, ' tipo: ', type(texto))
            palabras = texto.split()
            #print("PALABRAS:",palabras, ' tipo: ', type(palabras))
            if len(palabras) < 20 or len(palabras) > 100:  # descartamos párrafos cortos
                continue
            if texto:
                parrafos.append(texto)

  df = pd.DataFrame({'parrafo': parrafos})
  df.to_csv("libro_parrafos.csv", index=False, encoding="utf-8")

  print(f"Se extrajeron {len(parrafos)} párrafos y se guardaron en 'libro_parrafos.csv'.")

  f = pd.read_csv('libro_parrafos.csv')
  parrafos = pd.DataFrame(columns=['default', 'limpio'])
  parrafos['limpio'] = df['parrafo'].apply(normalize_text_X)
  parrafos['default'] = df['parrafo'].apply(normalize_text_y)

  return parrafos

In [34]:
def categoria_gramatical_stanza(palabra):
    doc = nlp(palabra)
    token = doc.sentences[0].words[0]
    return token.upos

upos2id = {
    "NOUN": 0, #Sustantivo común. Ej: gato, casa, libro, profesor
    "PROPN": 1, #Sustantivo propio. Ej: Argentina, Azul, Google
    "VERB": 2, #Verbo léxico. Ej: comer, hablar, correr
    "ADJ": 3, #Adjetivo. Ej: rápido, azul, brillante
    "ADV": 4, #Adverbio. Ej: rápidamente, muy, cerca
    "PRON": 5, #Pronombre. Ej: yo, tú, él, eso, alguien
    "DET": 6, #Determinante / artículo. Ej: el, la, los, un, ese, mi
    "ADP": 7, #Adposición: preposición o posposición. Ej: de, para, con, sin, sobre
    "SCONJ": 8, #Conjunción subordinante. Ej: que, porque, aunque, si
    "CCONJ": 9, #Conjunción coordinante. Ej: y, o, pero, ni
    "NUM": 10, #Numeral. Ej: uno, dos, 50, tercero
    "INTJ": 11, #Interjección. Ej: ay!, hola!, uf, eh
    "PART": 12, #Partícula gramatical (raro en español). Ejemplos típicos en inglés (not, 's), en español casi no se usa, pero aparece en casos como "sí" enfático.
    "AUX": 13, #Verbo auxiliar. Ej: haber, ser (cuando forman tiempos compuestos: “he comido”, “está hablando”)
    "PUNCT": 14, #Signos de puntuación. Ej: , . ; ! ?
    "SYM": 15, #Símbolos. Ej: $, %, +, =, →
    "X": 16 #Otros / desconocidos / extranjeros. Cualquier cosa que no encaja en ninguna categoría.
}

def indice_categoria_stanza(palabra):
    pos = categoria_gramatical_stanza(palabra)
    return upos2id.get(pos,-1)

In [35]:
def crearDataSetRFSinEtiquetas(parrafo):
  data_set_RF_sin_etiquetas = pd.DataFrame(columns = ['instancia_id', 'token', 'token_id', 'posicion_frase',
                                        'categoria_gramatical', 'distancia_al_final',
                                        'id_anterior', 'id_siguiente', 'es_principio',
                                        'es_medio', 'es_final', 'forma_parte'])

  instancia_ids = []
  tokens = []
  token_ids = []
  posiciones_parrafo = []
  categorias_gramaticales = []
  distancias_al_final = []
  ids_anteriores = []
  ids_siguientes = []
  son_principio = []
  son_medio = []
  son_final = []
  forman_parte = []

  for k, parrafo in enumerate(parrafo):
    instancia_id = k
    token_siguiente = -1
    token_anterior = -1
    palabras = parrafo.split()

    inicio_pregunta = False

    for i, palabra in enumerate(palabras):
      #categoria = indice_categoria_stanza(palabra)
      categoria = 0
      tokens_de_palabra = tokenizer.tokenize(palabra)

      for j, token in enumerate(tokens_de_palabra):
        id = tokenizer.convert_tokens_to_ids(token)
        tokens.append(token)
        token_ids.append(id)
        posiciones_parrafo.append(i)
        categorias_gramaticales.append(categoria)
        distancias_al_final.append(len(palabras) - i)

        ids_anteriores.append(token_anterior)
        token_anterior = id

        n_tok = len(tokens_de_palabra)
        if j != n_tok - 1:
          id_sig  = tokenizer.convert_tokens_to_ids(tokens_de_palabra[j + 1])
          ids_siguientes.append(id_sig)
        else:
          if i != len(palabras) - 1:
            token_siguiente = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(palabras[i + 1]))[0]
            ids_siguientes.append(token_siguiente)
          else:
            ids_siguientes.append(-1)

        es_medio_id = 0
        es_final_id = 0
        es_principio_id = 0
        if j == 0:
          es_principio_id = 1
        elif j == n_tok - 1:
          es_final_id = 1
        else:
          es_medio_id = 1
        son_principio.append(es_principio_id)
        son_medio.append(es_medio_id)
        son_final.append(es_final_id)

        forma_parte_id = 0
        if n_tok != 1:
          forma_parte_id = 1
        forman_parte.append(forma_parte_id)

        instancia_ids.append(instancia_id)

  data_set_RF_sin_etiquetas['instancia_id'] = instancia_ids
  data_set_RF_sin_etiquetas['token'] = tokens
  data_set_RF_sin_etiquetas['token_id'] = token_ids
  data_set_RF_sin_etiquetas['posicion_frase'] = posiciones_parrafo
  data_set_RF_sin_etiquetas['categoria_gramatical'] = categorias_gramaticales
  data_set_RF_sin_etiquetas['distancia_al_final'] = distancias_al_final
  data_set_RF_sin_etiquetas['id_anterior'] = ids_anteriores
  data_set_RF_sin_etiquetas['id_siguiente'] = ids_siguientes
  data_set_RF_sin_etiquetas['es_principio'] = son_principio
  data_set_RF_sin_etiquetas['es_medio'] = son_medio
  data_set_RF_sin_etiquetas['es_final'] = son_final
  data_set_RF_sin_etiquetas['forma_parte'] = forman_parte

  return data_set_RF_sin_etiquetas

In [175]:
def crearDataSetRFConEtiquetas(parrafos):
  data_set_RF_con_etiquetas = pd.DataFrame(columns = ['palabra_default', 'instancia_id', 'token', 'token_id', 'posicion_frase',
                                        'categoria_gramatical', 'distancia_al_final',
                                        'id_anterior', 'id_siguiente', 'es_principio',
                                        'es_medio', 'es_final', 'forma_parte', 'punt_inicial', 'punt_final', 'capitalización'])

  palabras_default = []

  instancia_ids = []
  tokens = []
  token_ids = []
  posiciones_parrafo = []
  categorias_gramaticales = []
  distancias_al_final = []
  ids_anteriores = []
  ids_siguientes = []
  son_principio = []
  son_medio = []
  son_final = []
  forman_parte = []
  puntuaciones_iniciales = []
  puntuaciones_finales = []
  capitalizaciones = []

  datos_limpios = parrafos['limpio']
  datos_default = parrafos['default']
  for k, parrafo in enumerate(datos_limpios):
    instancia_id = k
    token_siguiente = -1
    token_anterior = -1
    palabras = parrafo.split()

    inicio_pregunta = False

    palabras_d = datos_default.iloc[k].split()
    palabras_d = [subpalabra
                for palabra in palabras_d
                for subpalabra in (palabra.split('.') if ('.' in palabra and palabra[-1] != '.') else [palabra])]
    npal = 0
    for i, palabra in enumerate(palabras):
      palabra_default = ' '


      if palabra == "'": #or palabra == 'richdad' or palabra == 'www' or palabra == 'com' or palabra == 'infolibros' or palabra == 'org':
        continue

      #if palabras_d[npal] == 'www.richdad.com' or palabras_d[npal] == 'richdad.com' or palabras_d[npal] == 'InfoLibros.org':
      #  npal += 1




      if npal == len(palabras_d)-1:
        print(palabras_d, npal)
        print(instancia_id)
      while palabras_d[npal] == '?' or palabras_d[npal] == '¿' or palabras_d[npal] == '.' or palabras_d[npal] == ',' :
        if palabras_d[npal] == '¿':
          inicio_pregunta = True
        npal += 1

      if palabras_d[npal] != '?' and palabras_d[npal] != '¿' and palabras_d[npal] != '.' and palabras_d[npal] != ',':
        palabra_default = palabras_d[npal]

      if palabra_default[0] == "¿":
        inicio_pregunta = True

      if palabra_default[-1] == "?":
        puntuacion_final = 3
      elif palabra_default[-1] == ".":
        puntuacion_final = 1
      elif palabra_default[-1] == ",":
        puntuacion_final = 2
      else:
        if npal != len(palabras_d)-1:
          if palabras_d[npal+1] == '?':
            puntuacion_final = 3
          elif palabras_d[npal+1] == '.':
            puntuacion_final = 1
          elif palabras_d[npal+1] == ',':
            puntuacion_final = 2
          else:
            puntuacion_final = 0
        else:
          puntuacion_final = 0

      #categoria = indice_categoria_stanza(palabra)
      categoria = 0
      tokens_de_palabra = tokenizer.tokenize(palabra)

      if palabra_default.islower():
        capitalizacion = 0
      elif palabra_default.istitle():
        capitalizacion = 1
      elif palabra_default.isupper():
        capitalizacion = 3
      else:
        capitalizacion = 2

      npal += 1
      for j, token in enumerate(tokens_de_palabra):
        id = tokenizer.convert_tokens_to_ids(token)
        tokens.append(token)
        token_ids.append(id)
        posiciones_parrafo.append(i)
        categorias_gramaticales.append(categoria) #cambiar
        distancias_al_final.append(len(palabras) - i)

        ids_anteriores.append(token_anterior)
        token_anterior = id

        n_tok = len(tokens_de_palabra)
        if j != n_tok - 1:
          id_sig  = tokenizer.convert_tokens_to_ids(tokens_de_palabra[j + 1])
          ids_siguientes.append(id_sig)
        else:
          if i != len(palabras) - 1:
            token_siguiente = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(palabras[i + 1]))[0]
            ids_siguientes.append(token_siguiente)
          else:
            ids_siguientes.append(-1)

        es_medio_id = 0
        es_final_id = 0
        es_principio_id = 0
        if j == 0:
          es_principio_id = 1
        elif j == n_tok - 1:
          es_final_id = 1
        else:
          es_medio_id = 1
        son_principio.append(es_principio_id)
        son_medio.append(es_medio_id)
        son_final.append(es_final_id)

        forma_parte_id = 0
        if n_tok != 1:
          forma_parte_id = 1
        forman_parte.append(forma_parte_id)

        if inicio_pregunta:
          puntuaciones_iniciales.append(1)
          inicio_pregunta = False
        else:
          puntuaciones_iniciales.append(0)

        if j == n_tok - 1:
          puntuaciones_finales.append(puntuacion_final)
        else:
          puntuaciones_finales.append(0)
        print(palabra_default)
        capitalizaciones.append(capitalizacion)
        instancia_ids.append(instancia_id)
        palabras_default.append(palabra_default)

  data_set_RF_con_etiquetas['palabra_default'] = palabras_default
  data_set_RF_con_etiquetas['instancia_id'] = instancia_ids
  data_set_RF_con_etiquetas['token'] = tokens
  data_set_RF_con_etiquetas['token_id'] = token_ids
  data_set_RF_con_etiquetas['posicion_frase'] = posiciones_parrafo
  data_set_RF_con_etiquetas['categoria_gramatical'] = categorias_gramaticales
  data_set_RF_con_etiquetas['distancia_al_final'] = distancias_al_final
  data_set_RF_con_etiquetas['id_anterior'] = ids_anteriores
  data_set_RF_con_etiquetas['id_siguiente'] = ids_siguientes
  data_set_RF_con_etiquetas['es_principio'] = son_principio
  data_set_RF_con_etiquetas['es_medio'] = son_medio
  data_set_RF_con_etiquetas['es_final'] = son_final
  data_set_RF_con_etiquetas['forma_parte'] = forman_parte
  data_set_RF_con_etiquetas['punt_inicial'] = puntuaciones_iniciales
  data_set_RF_con_etiquetas['punt_final'] = puntuaciones_finales
  data_set_RF_con_etiquetas['capitalización'] = capitalizaciones

  return data_set_RF_con_etiquetas

# Carga de datos y creación de data set

In [ ]:
path = 'https://raw.githubusercontent.com/AzulBarr/Aprendizaje-Automatico/main/TPs/tp2'
libro1 = '/Harry_Potter_y_el_caliz_de_fuego_J_K_Rowling.epub'
path = path + libro1

In [ ]:
!wget -O libro1.epub $path

In [ ]:
parrafos = convertir_epub_a_pd('libro1.epub')

Data set sin categoria gramatical

In [ ]:
dataSet_RF_con_etiquetas = crearDataSetRFConEtiquetas(parrafos)
dataSet_RF_sin_etiquetas = crearDataSetRFSinEtiquetas(parrafos['limpio'])

In [ ]:
#dataSet_RF_con_etiquetas = crearDataSetRFConEtiquetas(parrafos)
#dataSet_RF_sin_etiquetas = crearDataSetRFSinEtiquetas(parrafos['limpio'])

Data set con categoria gramatical

In [ ]:
path = 'https://raw.githubusercontent.com/AzulBarr/Aprendizaje-Automatico/main/TPs/tp2'
datos = '/dataSet_RF_con_etiquetas.csv'
path = path + datos

In [ ]:
!wget -O dataSet_RF_con_etiquetas.csv $path

In [ ]:
dataSet_RF_con_etiquetas_cg = pd.read_csv('dataSet_RF_con_etiquetas.csv')

## Etiqueta 1: capitalización
* 0: todo en minúsculas (ej: “hola”)
* 1: primera letra en mayúscula (ej: “Hola”) (incluye palabras de 1 letra)
* 2: algunas (pero no todas) letras en mayúscula (ej: “McDonald's”  “iPhone”)
* 3: todo en mayúsculas (ej.: “ONU”, “NASA”, “UBA”) (más de una letra)

## Etiqueta 2: puntuación inicial
* 0: sin puntuación.
* 1: "?"

## Etiqueta 3: puntuación final
* 0: sin puntuación.
* 1: "."
* 2: ","
* 3: "?"

# Separación en train y test

In [ ]:
palabras_d = ['grandioso y solitario paisaje, o andar por los montes como res herida, espantando zancudos, sin perjuicio de que Mayo se llene de nuches..., pobre'].split()
    palabras_d = [subpalabra
                for palabra in palabras_d
                for subpalabra in (palabra.split('.') if ('.' in palabra and palabra[-1] != '.') else [palabra])]

In [160]:
parrafos_final = pd.read_csv('dataset_parrafos_total3.csv')
parrafos_final = parrafos_final.dropna()
parrafos_final['limpio'] = parrafos_final['limpio'].apply(normalize_text_X)
parrafos_final['default'] = parrafos_final['default'].apply(normalize_text_y)
parrafos_final.reset_index(drop=True)
parrafos_final.drop(index=[7764, 9497, 9498, 13804], inplace=True) #palabra.¿palabra parrafo raro, parrafo raro, palabra.palabra
parrafos_final.reset_index(drop=True)

,default,limpio
0,Entonces es que es así como debo llamarme repu...,entonces es que es así como debo llamarme repu...
1,"Pero, Heidi, las cosas no son tan simples",pero heidi las cosas no son tan simples
2,¿Entonces qué es?,entonces qué es
3,¿No te he prohibido terminantemente que vayas ...,no te he prohibido terminantemente que vayas a...
4,"Y luego, cuando nos encontramos desgraciados y...",y luego cuando nos encontramos desgraciados y ...
...,...,...
243606,Soy Donna.,soy donna
243607,"Cuidado con llamarme pequeño, o te",cuidado con llamarme pequeño o te
243608,La mitad se puso de pie.,la mitad se puso de pie
243609,Tienes una acreción de placenta.,tienes una acreción de placenta


In [179]:
parrafos_final[1555:1556] #9497, 9498

,default,limpio
1555,"grandioso y solitario paisaje, o andar por los...",grandioso y solitario paisaje o andar por los ...


In [178]:
dataSet_RF_con_etiquetas_final = crearDataSetRFConEtiquetas(parrafos_final[1555:1556])

grandioso
grandioso
y
solitario
paisaje,
paisaje,
o
andar
andar
por
los
montes
como
res
herida,
herida,
espantando
espantando
espantando
zancudos,
zancudos,
zancudos,
sin
perjuicio
perjuicio
perjuicio
de
que
Mayo
se
llene
llene
de
nuches
nuches


IndexError: string index out of range

In [177]:
dataSet_RF_con_etiquetas_final

,palabra_default,instancia_id,token,token_id,posicion_frase,categoria_gramatical,distancia_al_final,id_anterior,id_siguiente,es_principio,es_medio,es_final,forma_parte,punt_inicial,punt_final,capitalización
0,¿Es,0,es,10196,0,0,5,-1,14104,1,0,0,0,1,0,1
1,tiempo,0,tiempo,14104,1,0,4,10196,10104,1,0,0,0,0,0,0
2,de,0,de,10104,2,0,3,14104,10109,1,0,0,0,0,0,0
3,la,0,la,10109,3,0,2,10104,10347,1,0,0,0,0,0,0
4,bebida?,0,be,10347,4,0,1,10109,33341,1,0,0,1,0,0,0
5,bebida?,0,##bida,33341,4,0,1,10347,-1,0,0,1,1,0,3,0


In [ ]:
train = dataSet_RF_con_etiquetas[dataSet_RF_con_etiquetas['instancia_id'] <= 2972]
test = dataSet_RF_con_etiquetas[dataSet_RF_con_etiquetas['instancia_id'] > 2972]

In [ ]:
train_cg = dataSet_RF_con_etiquetas_cg[dataSet_RF_con_etiquetas_cg['instancia_id'] <= 2972]
test_cg = dataSet_RF_con_etiquetas_cg[dataSet_RF_con_etiquetas_cg['instancia_id'] > 2972]

In [ ]:
X_train = train.drop(columns=['palabra_default', 'token', 'categoria_gramatical','capitalización', 'punt_inicial', 'punt_final'])
y_train = train[['capitalización', 'punt_inicial', 'punt_final']]
X_test = test.drop(columns=['palabra_default', 'token', 'categoria_gramatical', 'capitalización', 'punt_inicial', 'punt_final'])
y_test = test[['capitalización', 'punt_inicial', 'punt_final']]

In [ ]:
X_train_cg = train_cg.drop(columns=['palabra_default', 'token', 'capitalización', 'punt_inicial', 'punt_final'])
y_train_cg = train_cg[['capitalización', 'punt_inicial', 'punt_final']]
X_test_cg = test_cg.drop(columns=['palabra_default', 'token',  'capitalización', 'punt_inicial', 'punt_final'])
y_test_cg = test_cg[['capitalización', 'punt_inicial', 'punt_final']]

# Sin etiqueta gramatical


## Random Forest para etiqueta 1

In [ ]:
y_train1 = y_train['capitalización']
y_test1 = y_test['capitalización']

In [ ]:
model = RandomForestClassifier(n_estimators=200, max_depth=None, random_state=42, min_samples_split=10)
model.fit(X_train, y_train1)

In [ ]:
y_pred1 = model.predict(X_test)

In [ ]:
f1_score(y_test1, y_pred1, average = "macro")

In [ ]:
cm1 = confusion_matrix(y_test1,y_pred1, labels=[0,1,2,3])

In [ ]:
sns.heatmap(cm1, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicho")
plt.ylabel("Real")
plt.title("Matriz de confusión para capitalización sin CG")   # ← Título

plt.show()


## Random Forest para etiqueta 2

In [ ]:
y_train2 = y_train['punt_inicial']
y_test2 = y_test['punt_inicial']

In [ ]:
model = RandomForestClassifier(n_estimators=200, max_depth=None, random_state=42, min_samples_split=10)
model.fit(X_train, y_train2)

In [ ]:
y_pred2 = model.predict(X_test)

In [ ]:
f1_score(y_test2, y_pred2, average = "macro")

In [ ]:
cm2 = confusion_matrix(y_test2, y_pred2, labels=[0,1])

In [ ]:
sns.heatmap(cm2, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicho")
plt.ylabel("Real")
plt.title('Matriz de confusión para punt_inicial sin CG')

plt.show()

## Random Forest para etiqueta 3

In [ ]:
y_train3 = y_train['punt_final']
y_test3 = y_test['punt_final']

In [ ]:
model = RandomForestClassifier(n_estimators=200, max_depth=None, random_state=42, min_samples_split=10)
model.fit(X_train, y_train3)

In [ ]:
y_pred3 = model.predict(X_test)

In [ ]:
f1_score(y_test3, y_pred3, average = "macro")

In [ ]:
cm3 = confusion_matrix(y_test3, y_pred3, labels=[0,1,2,3])

In [ ]:
sns.heatmap(cm3, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicho")
plt.ylabel("Real")
plt.title('Matriz de confusión para punt_final sin CG')
plt.show()

# Con etiqueta gramatical


## Random Forest para etiqueta 1

In [ ]:
y_train4_cg = y_train_cg['capitalización']
y_test4_cg = y_test_cg['capitalización']

In [ ]:
model = RandomForestClassifier(n_estimators=200, max_depth=None, random_state=42, min_samples_split=10)
model.fit(X_train_cg, y_train4_cg)

In [ ]:
y_pred4_cg = model.predict(X_test_cg)

In [ ]:
f1_score(y_test4_cg, y_pred4_cg, average = "macro")

In [ ]:
cm4 = confusion_matrix(y_test4_cg, y_pred4_cg, labels=[0,1,2,3])

In [ ]:
sns.heatmap(cm4, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicho")
plt.ylabel("Real")
plt.title('Matriz de confusión para capitalización con CG')

plt.show()

## Random Forest para etiqueta 2

In [ ]:
y_train5_cg = y_train_cg['punt_inicial']
y_test5_cg = y_test_cg['punt_inicial']

In [ ]:
model = RandomForestClassifier(n_estimators=200, max_depth=None, random_state=42, min_samples_split=10)
model.fit(X_train_cg, y_train5_cg)

In [ ]:
y_pred5_cg = model.predict(X_test_cg)

In [ ]:
f1_score(y_test5_cg, y_pred5_cg, average = "macro")

In [ ]:
cm5 = confusion_matrix(y_test5_cg, y_pred5_cg, labels=[0,1,2,3])

In [ ]:
sns.heatmap(cm5, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicho")
plt.ylabel("Real")
plt.title('Matriz de confusión para punt_inicial con CG')

plt.show()

## Random Forest para etiqueta 3

In [ ]:
y_train6_cg = y_train_cg['punt_final']
y_test6_cg = y_test_cg['punt_final']

In [ ]:
model = RandomForestClassifier(n_estimators=200, max_depth=None, random_state=42, min_samples_split=10)
model.fit(X_train_cg, y_train6_cg)

In [ ]:
y_pred6_cg = model.predict(X_test_cg)

In [ ]:
f1_score(y_test6_cg, y_pred6_cg, average = "macro")

In [ ]:
cm6 = confusion_matrix(y_test6_cg, y_pred6_cg, labels=[0,1,2,3])

In [ ]:
sns.heatmap(cm6, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicho")
plt.ylabel("Real")
plt.title('Matriz de confusión para punt_final con CG')
plt.show()